In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import re
import csv

In [2]:
CATEGORIES = ['Elektronik & Computer', 'Haushalt, Garten, Baumarkt', 'Beauty, Drogerie & Lebensmittel', 'Spielzeug & Baby', 'Kleidung, Schuhe & Uhren', 'Sport & Freizeit',
              'Auto, Motorrad & Gewerbe']

In [5]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
            and content_type is not None
            and content_type.find('html') > -1)

def log_error(e):
    """
    It is always a good idea to log errors.
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [6]:
raw_html = simple_get('https://smile.amazon.de/gp/site-directory/ref=nav_shopall_btn')
html = BeautifulSoup(raw_html, 'html.parser')
category_table = html.find('table',  {"id": "shopAllLinks"})
big_category_div = category_table.find_all('div', class_='popover-grouping')

In [7]:
all_categories = []
for i, div in enumerate(big_category_div):
    if div.find('h2').text in CATEGORIES:
        for link in div.find_all('a'):
            if 'node' in link['href']:
                all_categories.append(link['href'])

In [12]:
link = all_categories[1]
full_url = 'https://www.amazon.de' + link
page = BeautifulSoup(simple_get(full_url), 'html.parser')

In [16]:
products = page.find_all('a')

In [29]:
def get_product_link(url):
    pattern_numbers_strings = re.compile(r'\/[A-Z0-9]{10}')
    pattern_numbers = re.compile(r'[0-9]+')
    pattern_strings = re.compile(r'[a-zA-Z]+')
    match = pattern_numbers_strings.findall(url)
    if match:
        if pattern_numbers.findall(match[0]):
            if pattern_strings.findall(match[0]):
                return match[0]
    return None


In [31]:
product_ids = []
for i, paragraph in enumerate(products):
    if paragraph.has_attr('href'):
        id = get_product_link(paragraph['href'])
        if id:
            product_ids.append(id)
set(product_ids)

{'/B00J0S6QZ6',
 '/B00LN803LE',
 '/B00WLI5E3M',
 '/B01BBWWFB2',
 '/B01BTZFM0W',
 '/B01CU1EC6Y',
 '/B01EY78FQU',
 '/B01GJC4WRO',
 '/B01MCW7EOW',
 '/B01MPWZ4M7',
 '/B06XCJP94G',
 '/B06XJ49G5B',
 '/B07215Q3Y1',
 '/B0725WZK1N',
 '/B0756BRVXJ',
 '/B0756CYWWD',
 '/B075LXNGFH',
 '/B0787SZTQV',
 '/B07BHDC9V6',
 '/B07G4M421W',
 '/B07G4P7MVG',
 '/B07G4PKF7S',
 '/B07G4SZ27D',
 '/B07GPQWZQ2',
 '/B07GPR11N1',
 '/B07GQ4H6KX',
 '/B07GQ6HTCN',
 '/B07GQ6KPRJ',
 '/B07GQ6L7S3',
 '/B07GQ6YMXC',
 '/B07GQ6Z4JQ'}

In [ ]:
for i, paragraph in enumerate(products):
    if paragraph.has_attr('href'):
        id = get_product_link(paragraph['href'])
        print(id)
        if id:
            if not id in product_ids:
                product_ids.append(id)
                writer.writerow(id)
                print(i, ' ', paragraph.text)


In [39]:
import os
products = []
path = 'C:\\Users\\Josef\\PycharmProjects\\QC-Yes-No\\Tweet extraction\\products\\'
for file in os.listdir(path):
    with open(path + file, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        for row in reader:
            products.append(row[0])
len(set(products))

3897